In [2]:
# %load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

from sales_data import SalesData
from reg import LinReg, PolyReg, MultiReg
plt.style.use('ggplot')

# Import Data and Format Columns
... then aggregate data to monthly.

In [3]:
sd = SalesData()
sd.set()

FileNotFoundError: [Errno 2] File data/shops.csv does not exist: 'data/shops.csv'

# Sales overall by month

In [ ]:
def plot_trends(df, title_id):
    x = df['year_month']
    y1 = df['item_cnt_month']
    y2 = df['sales_month']

    fig, ax1 = plt.subplots(figsize=(16, 8))
    ax2 = ax1.twinx()
    ax1.plot(x, y1, 'g-', label='Item Count')
    ax2.plot(x, y2, 'b-', label='Sales')
    
    ax1.set_xlabel('Period')
    ax1.tick_params(axis='x', rotation=90)
    ax1.set_ylabel('Item Count')
    ax2.set_ylabel('Sales')
    fig.legend()
#     ax2.legend(loc='best')
    plt.title(f'{title_id}\nItem Count & Sales')
    plt.show()

In [ ]:
sd.monthly_agg()
sd.calc_sales()
sales = sd.monthly_sales

In [ ]:
overall = sales.groupby(['year_month'])[['item_cnt_month', 'sales_month']].sum().reset_index()
overall.sort_values(['year_month'], inplace=True)
plot_trends(overall, 'Overall')

## Sales by City

In [ ]:
sd.add_shop_cat_info()

In [ ]:
sales = sd.monthly_sales

In [ ]:
gb_cols = ['loc_name', 'year_month']
city_sales = sales.groupby(gb_cols)[['item_cnt_month', 'sales_month']].sum().reset_index()

for n in sorted(city_sales['loc_name'].unique()):
    print(f'********\n{n}\n********\n')
    subdf = city_sales.loc[city_sales['loc_name'] == n]
    print(subdf.shape)
    plot_trends(subdf, n)

## Sales by the location types

In [ ]:
loc_types = ['тц', 'трц', 'мега', 'тк', 'трк', 'молл', 'центральный']

for t in loc_types:
    print(f'********\n{t}\n********\n')
    subdf = sales.loc[sales[t] == 1]
    subdf = subdf.groupby(['year_month'])[['item_cnt_month', 'sales_month']].sum().reset_index()
    plot_trends(subdf, t)

# Focus on a Single Item, Shop

In [ ]:
sd.add_item_info()

sales = sd.monthly_sales

In [ ]:
# top_majorcat = 'Игры'
top_loc = 'МОСКВА'
top_loc_type = 'тц'
top_shop_id = '31'
top_cat = 'Игры - XBOX 360'
top_item_id = '4870'

# majorcat_mask = sales['major_category'] == top_majorcat
loc_mask = sales['loc_name'] == top_loc
shop_mask = sales['shop_id'] == top_shop_id
cat_mask = sales['item_category_name'] == top_cat
item_mask = sales['item_id'] == top_item_id

top_item = sales.loc[
#                        majorcat_mask & 
                       loc_mask & 
                       shop_mask & 
                       cat_mask &
                       item_mask &
                       (sales[top_loc_type] == 1)]
top_item.head()

In [ ]:
plot_trends(top_item, 'Top Location & Item')

### Item Counts in Top Category

In [ ]:
top_cat = sales.loc[
#                        majorcat_mask & 
                       loc_mask & 
                       shop_mask & 
                       cat_mask &
#                        item_mask &
                       (sales[top_loc_type] == 1)]

In [ ]:
top_cat.groupby('item_id')['item_cnt_month'] \
    .sum().reset_index().sort_values('item_cnt_month', ascending=False).head(20)

## Focus on a single shop/item

In [ ]:
top_shop_id = '31'
top_item_id = '3328'

top_item = sales.loc[(sales['shop_id'] == top_shop_id) & 
                    (sales['item_id'] == top_item_id)].copy()

plot_trends(top_item, 'Top Shop/Item')

## Decompose

In [ ]:
def plot_seasonal_decomposition(axs, series, sd):
    axs[0].plot(series.index, series)
    axs[0].set_title("Raw Series")
    axs[1].plot(series.index, sd.trend)
    axs[1].set_title("Trend Component $T_t$")
    axs[2].plot(series.index, sd.seasonal)
    axs[2].set_title("Seasonal Component $S_t$")
    axs[3].plot(series.index, sd.resid)
    axs[3].set_title("Residual Component $R_t$")
    
def plot_series_and_difference(axs, series, title):
    diff = series.diff()
    axs[0].plot(series.index, series)
    axs[0].set_title("Raw Series: {}".format(title))
    axs[1].plot(series.index, diff)
    axs[1].set_title("Series of First Differences: {}".format(title))
    
def format_list_of_floats(L):
    return ["{0:2.2f}".format(f) for f in L]

In [ ]:
# format plot_date as datetime
top_item['plot_date'] = top_item['month'].astype(str) + '-' + top_item['year'].astype(str)
top_item['plot_date'] = top_item['plot_date'].apply(lambda x: datetime.strptime(x, '%m-%Y'))

topitem_trend = top_item.loc[:, ['plot_date', 'item_cnt_month']].copy()
topitem_trend = topitem_trend.set_index('plot_date')

In [ ]:
topitem_trend = sd.add_missing_periods(topitem_trend)

In [ ]:
python_decomposition = sm.tsa.seasonal_decompose(topitem_trend)
fig, axs = plt.subplots(4, figsize=(14, 8))
plot_seasonal_decomposition(axs, topitem_trend, python_decomposition)
plt.tight_layout()

### Test for Stationarity

In [ ]:
test = sm.tsa.stattools.adfuller(topitem_trend)
print(f'ADF p-value for topitem_trend: {np.round(test[1], 2)}')

In [ ]:
topitem_diff = topitem_trend.diff()[1:]
test = sm.tsa.stattools.adfuller(topitem_diff)
print(f'ADF p-value for topitem_trend: {np.round(test[1], 2)}')

The differenced series is stationary.

In [ ]:
fig, axs = plt.subplots(2, figsize=(14, 4))
plot_series_and_difference(axs, topitem_diff, 'Top Item')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 3))
_ = sm.graphics.tsa.plot_acf(topitem_trend, lags=25, ax=ax)

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 3))
_ = sm.graphics.tsa.plot_pacf(topitem_trend, lags=25, ax=ax)

In [ ]:
topitem_model = ARIMA(topitem_trend, order=(2, 1, 0)).fit()

In [ ]:
print("ARIMA(3, 1, 0) coefficients from Top Item model:\n  Intercept {0:2.2f}\n  AR {1}".format(
    topitem_model.params[0], 
        format_list_of_floats(list(topitem_model.params[1:]))
    ))

## Predict

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 4))
ax.plot(topitem_trend)
fig = topitem_model.plot_predict(start='2015-10-01', end='2015-12-01', 
                                 dynamic=True, ax=ax, plot_insample=False)

Not enough points to fit an ARIMA model properly?

# Regressions
Look at both linear and polynomial regressions...

In [ ]:
l = [
    {'ID': 0, 'item_cnt_month': 0.1},
    {'ID': 1, 'item_cnt_month': 0.2},
    {'ID': 2, 'item_cnt_month': 0.3},
    {'ID': 3, 'item_cnt_month': 0.4},
]

pd.DataFrame.from_records(l)

In [ ]:
topitem_trend.tail()

In [ ]:
# X = np.arange(topitem_trend.shape[0] - 1).reshape(-1, 1)
X = topitem_trend.index
y = topitem_trend.iloc[:, 0]
# plt.plot(X, y)

mr = MultiReg(poly_degree=2)
mr.fit(X, y)
y_pred = mr.predict_next()

print(f'Predicted Value for 201511: {y_pred}')

mr.plot()

This is worth trying out. 

Need to iterate through each of the test cases to find the shop/item combo and do the regression... is this the best way to do this?

In [5]:
test_set = pd.read_csv('../data/test.csv')
# test_set.insert(loc=1, column='month', value='11')
test_set['shop_id'] = test_set['shop_id'].astype(str)
test_set['item_id'] = test_set['item_id'].astype(str)
print(test_set.shape)

(214200, 3)


In [6]:
test_set.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [7]:
test_set['shop_id'].value_counts()

12    5100
36    5100
49    5100
39    5100
52    5100
45    5100
28    5100
34    5100
58    5100
22    5100
41    5100
57    5100
18    5100
25    5100
26    5100
42    5100
50    5100
10    5100
48    5100
55    5100
5     5100
37    5100
24    5100
56    5100
14    5100
15    5100
19    5100
21    5100
59    5100
47    5100
16    5100
44    5100
31    5100
46    5100
7     5100
2     5100
35    5100
3     5100
53    5100
38    5100
4     5100
6     5100
Name: shop_id, dtype: int64

In [ ]:
for row in range(test_set.shape[0]):
    row_id = test_set.iloc[row, 0]
    shop_id = test_set.iloc[row, 1]
    item_id = test_set.iloc[row, 2]
    print(row_id, shop_id, item_id)

## Item Categories
Most item categories appear to have a more general category description. Break this out into its own column. Then, merge categories to items.

In [ ]:
# split on hyphen
mask = cats['item_category_name'].str.contains(' -')
cats.loc[mask, 'major_category'] =  cats.loc[
    mask, 'item_category_name'].apply(lambda x: x.split(' -')[0])

# get description before parenthesis
mask1 = cats['item_category_name'].str.contains(' \(')
mask2 = cats['major_category'].isna()
cats.loc[mask1 & mask2, 'major_category'] =  cats.loc[
    mask1 & mask2, 'item_category_name'].apply(lambda x: x.split(' (')[0])

# whatever is left
mask = cats['major_category'].isna()
cats.loc[mask, 'major_category'] =  cats.loc[mask, 'item_category_name']

items = pd.merge(items, cats, on=['item_category_id'], how='inner')

In [ ]:
items.head()

## Sales by Major Categories

In [ ]:
sales = pd.merge(sales, items, on=['item_id'], how='inner')

for cat in sorted(sales['major_category'].unique()):
    print(f'**********\n{cat}\n**********')
    subdf = sales.loc[sales['major_category'] == cat]
    subdf = subdf.groupby('year_month')[['item_cnt_day', 'sales_day']].sum().reset_index()
    plot_trends(subdf, cat)
          

## Focus on Tops

In [ ]:
top_majorcat = 'Игры'
top_loc = 'МОСКВА'
top_loc_type = 'тц'
top_shop_id = 31
top_cat = 'Игры - XBOX 360'

In [ ]:
majorcat_mask = sales['major_category'] == top_majorcat
loc_mask = sales['loc_name'] == top_loc
shop_mask = sales['shop_id'] == top_shop_id
cat_mask = sales['item_category_name'] == top_cat

top_catloc = sales.loc[majorcat_mask & 
                       loc_mask & 
                       shop_mask & 
                       cat_mask &
                       (sales[top_loc_type] == 1)]
top_catloc.head()

In [ ]:
top_catloc['item_category_name'].unique()

In [ ]:
top_items = top_catloc.groupby(['item_name', 'item_id'])[['item_cnt_day', 'sales_day']].sum().reset_index()
top_items.sort_values(['item_cnt_day'], ascending=False, inplace=True)

In [ ]:
top_items.head(20)

In [ ]:
items.head()

In [ ]:
sd.items.loc[sd.items['item_id'] == '3342']

In [ ]:
top_catloc['item_name'].unique()

In [ ]:
top_catloc.columns

In [ ]:
for cat_name in sorted(top_catloc['item_category_name'].unique()):
    print(f'*********\n{cat_name}\n***********')
    subdf = top_catloc.loc[top_catloc['item_category_name'] == cat_name]
    subdf = subdf.groupby('year_month')[['item_cnt_day', 'sales_day']].sum().reset_index()
    plot_trends(subdf, cat_name)

In [ ]:
gb_cols = ['shop_id', 'shop_name', 'clean_name', 'loc_name',
           'тц', 'трц', 'мега', 'тк', 'трк', 'молл', 'центральный']
shop_agg = top_catloc.groupby(gb_cols)[['item_cnt_day', 'sales_day']].sum().reset_index()
shop_agg.sort_values('item_cnt_day', ascending=False, inplace=True)

In [ ]:
shop_agg.head()

In [ ]:
sales.head()

In [ ]:
sales.columns

In [ ]:
items['major_category'].value_counts()

In [ ]:
items.head()

In [ ]:
cats.loc[cats['major_category'] == 'Книги']

In [ ]:
items['item_category_id'].value_counts()

In [ ]:
cats

In [ ]:
items.loc[items['item_category_id'] == 40]

In [ ]:
items.head(20)

In [ ]:
cats.loc[40:, :]